In [1]:
!pip install python-bcb pandas seaborn sklearn yfinance xgboost optuna scikit-learn xgboost joblib

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [2]:
import pandas as pd
import seaborn as sns
import yfinance as yf
from bcb import sgs
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
import warnings
import joblib


c:\Users\alexi\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_INICIAL = '2019-04-09'
DATA_FIM = '2025-10-01'

In [4]:
tickers = ['^GSPC', '^VIX', '^TNX',  'BZ=F', 'BRL=X', 'IRFM11.SA']
dataframes = []
closes = []
for ticker in tickers:
    dado = yf.download(
        ticker,
        start= DATA_INICIAL,
        end= DATA_FIM,
        auto_adjust=True
    )

    if not dado.empty:
        close = dado[['Close']].rename(columns={'Close': ticker})
        closes.append(close)
df_final = pd.concat(closes, axis=1)
print(df_final)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price             ^GSPC       ^VIX   ^TNX       BZ=F     BRL=X  IRFM11.SA
Ticker            ^GSPC       ^VIX   ^TNX       BZ=F     BRL=X  IRFM11.SA
Date                                                                     
2019-04-09  2878.199951  14.280000  2.499  70.610001  3.848200        NaN
2019-04-10  2888.209961  13.300000  2.477  71.730003  3.848000        NaN
2019-04-11  2888.320068  13.020000  2.504  70.830002  3.824000        NaN
2019-04-12  2907.409912  12.010000  2.560  71.550003  3.855800        NaN
2019-04-15  2905.580078  12.320000  2.553  71.180000  3.880300        NaN
...                 ...        ...    ...        ...       ...        ...
2025-09-24  6637.970215  16.180000  4.147  69.309998  5.281600  93.750000
2025-09-25  6604.720215  16.740000  4.172  69.419998  5.329900  93.629997
2025-09-26  6643.700195  15.290000  4.187  70.129997  5.363700  93.660004
2025-09-29  6661.209961  16.120001  4.141  67.970001  5.342253  93.639999
2025-09-30  6688.459961  16.280001  4.

In [5]:
ipca_exp = sgs.get(433, start=DATA_INICIAL, end=DATA_FIM)
ipca_exp.rename(columns={433: 'IPCA_Expectativa_12m'}, inplace=True)

selic = sgs.get(432, start=DATA_INICIAL, end=DATA_FIM)
selic.rename(columns={432: 'Selic_Meta'}, inplace=True)

ibcbr = sgs.get(24363, start=DATA_INICIAL, end=DATA_FIM)
ibcbr.rename(columns={24363: 'IBC_Br'}, inplace=True)

ptax = sgs.get(1, start=DATA_INICIAL, end=DATA_FIM)
ptax.rename(columns={1: 'Dolar_PTAX'}, inplace=True)

macro_bcb = {
    'IPCA_EXP': ipca_exp,
    'SELIC': selic,
    'IBC_BR': ibcbr,
    'PTAX': ptax
}
macro_bcb['SELIC'].describe()

macro_df = pd.concat(macro_bcb.values(), axis=1)
macro_df.head()
macro_df = macro_df.ffill()
macro_df.head()

,433,432,24363,1
Date,,,,
2019-04-01,0.57,NaN,97.86275,NaN
2019-04-09,0.57,6.5,97.86275,3.8563
2019-04-10,0.57,6.5,97.86275,3.8345
2019-04-11,0.57,6.5,97.86275,3.8399
2019-04-12,0.57,6.5,97.86275,3.8685


In [6]:
join = [macro_df, df_final]

externo = pd.concat(join, axis=1)
print(externo)

             433   432      24363       1  (^GSPC, ^GSPC)  (^VIX, ^VIX)  \
Date                                                                      
2019-04-01  0.57   NaN   97.86275     NaN             NaN           NaN   
2019-04-09  0.57   6.5   97.86275  3.8563     2878.199951     14.280000   
2019-04-10  0.57   6.5   97.86275  3.8345     2888.209961     13.300000   
2019-04-11  0.57   6.5   97.86275  3.8399     2888.320068     13.020000   
2019-04-12  0.57   6.5   97.86275  3.8685     2907.409912     12.010000   
...          ...   ...        ...     ...             ...           ...   
2025-09-27  0.48  15.0  109.44199  5.3445             NaN           NaN   
2025-09-28  0.48  15.0  109.44199  5.3445             NaN           NaN   
2025-09-29  0.48  15.0  109.44199  5.3229     6661.209961     16.120001   
2025-09-30  0.48  15.0  109.44199  5.3186     6688.459961     16.280001   
2025-10-01  0.09  15.0  109.61024  5.3208             NaN           NaN   

            (^TNX, ^TNX)

In [7]:
url_focus = (
    "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/"
    "ExpectativasMercadoAnuais?$top=100000"
    "&$filter=Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2019-04-09'"
    "&$format=json"
    "&$orderby=Data%20asc"
)
try:
    json = pd.read_json(url_focus)
    Focus = pd.DataFrame(json['value'].tolist())

    Focus['Data'] = pd.to_datetime(Focus['Data'])

    Focus['DataReferencia'] = Focus['DataReferencia'].astype(int)
    
    Focus['Ano_Divulgacao'] = Focus['Data'].dt.year
    
    Focus = Focus[Focus['DataReferencia'] == Focus['Ano_Divulgacao']].copy()

    Focus = Focus.set_index('Data').sort_index()
    
    Focus = Focus[['Mediana']].rename(columns={'Mediana': 'IPCA_Expectativa_AnoCorrente'})
    
    print(Focus.tail())

except Exception as e:
    raise TypeError
print(Focus)

            IPCA_Expectativa_AnoCorrente
Data                                    
2025-12-17                        4.3371
2025-12-18                        4.3475
2025-12-18                        4.3313
2025-12-19                        4.3300
2025-12-19                        4.3163
            IPCA_Expectativa_AnoCorrente
Data                                    
2019-04-09                        3.9000
2019-04-09                        3.9100
2019-04-10                        4.1000
2019-04-10                        4.0600
2019-04-11                        4.0800
...                                  ...
2025-12-17                        4.3371
2025-12-18                        4.3475
2025-12-18                        4.3313
2025-12-19                        4.3300
2025-12-19                        4.3163

[3372 rows x 1 columns]


In [8]:
Focus = Focus.groupby(Focus.index).mean()
Focus = Focus.sort_index()
print(Focus)

            IPCA_Expectativa_AnoCorrente
Data                                    
2019-04-09                       3.90500
2019-04-10                       4.08000
2019-04-11                       4.05500
2019-04-12                       4.04500
2019-04-15                       4.03000
...                                  ...
2025-12-15                       4.34895
2025-12-16                       4.34805
2025-12-17                       4.34230
2025-12-18                       4.33940
2025-12-19                       4.32315

[1686 rows x 1 columns]


In [9]:
FeatureBase = pd.merge(externo, Focus, left_index=True, right_index=True, how='inner')
FeatureSemanal = FeatureBase[FeatureBase.index.dayofweek == 4].copy()
print(FeatureSemanal)

             433   432      24363       1  (^GSPC, ^GSPC)  (^VIX, ^VIX)  \
2019-04-12  0.57   6.5   97.86275  3.8685     2907.409912     12.010000   
2019-04-26  0.57   6.5   97.86275  3.9353     2939.879883     12.730000   
2019-05-03  0.13   6.5   97.74783  3.9388     2945.639893     12.870000   
2019-05-10  0.13   6.5   97.74783  3.9578     2881.399902     16.040001   
2019-05-17  0.13   6.5   97.74783  4.0844     2859.530029     15.960000   
...          ...   ...        ...     ...             ...           ...   
2025-08-29 -0.11  15.0  110.42518  5.4264     6460.259766     15.360000   
2025-09-05  0.48  15.0  109.44199  5.3968     6481.500000     15.180000   
2025-09-12  0.48  15.0  109.44199  5.3677     6584.290039     14.760000   
2025-09-19  0.48  15.0  109.44199  5.3276     6664.359863     15.450000   
2025-09-26  0.48  15.0  109.44199  5.3445     6643.700195     15.290000   

            (^TNX, ^TNX)  (BZ=F, BZ=F)  (BRL=X, BRL=X)  \
2019-04-12         2.560     71.550003   

In [10]:
finbert = pd.read_csv('FinBert.csv')
finbert['data'] = pd.to_datetime(finbert['data'])

Diario = finbert.groupby('data')['score'].sum().sort_index()
Diario = Diario.to_frame(name='scoreTotal')

Diario['scoreSemanal'] = Diario['scoreTotal'].rolling('7D', min_periods=1).mean()

FeatureB = Diario.loc[Diario.index.dayofweek == 4, ['scoreSemanal']].copy()

print(FeatureB.tail())

            scoreSemanal
data                    
2025-09-26      4.593504
2025-10-03      3.399397
2025-10-10      3.692898
2025-10-17      3.071258
2025-10-24      3.468857


In [11]:
Features_B = pd.merge(FeatureB, FeatureSemanal, left_index=True, right_index=True, how='inner')
print(Features_B)

            scoreSemanal   433   432      24363       1  (^GSPC, ^GSPC)  \
2019-04-12      1.072554  0.57   6.5   97.86275  3.8685     2907.409912   
2019-04-26      1.186250  0.57   6.5   97.86275  3.9353     2939.879883   
2019-05-03      1.419986  0.13   6.5   97.74783  3.9388     2945.639893   
2019-05-10      1.171628  0.13   6.5   97.74783  3.9578     2881.399902   
2019-05-17      0.217869  0.13   6.5   97.74783  4.0844     2859.530029   
...                  ...   ...   ...        ...     ...             ...   
2025-08-29      2.698916 -0.11  15.0  110.42518  5.4264     6460.259766   
2025-09-05      1.993150  0.48  15.0  109.44199  5.3968     6481.500000   
2025-09-12      2.419663  0.48  15.0  109.44199  5.3677     6584.290039   
2025-09-19      3.556344  0.48  15.0  109.44199  5.3276     6664.359863   
2025-09-26      4.593504  0.48  15.0  109.44199  5.3445     6643.700195   

            (^VIX, ^VIX)  (^TNX, ^TNX)  (BZ=F, BZ=F)  (BRL=X, BRL=X)  \
2019-04-12     12.010000   

In [12]:
nans = Features_B.isna().sum()
print(nans[nans > 0])

(^GSPC, ^GSPC)             3
(^VIX, ^VIX)               3
(^TNX, ^TNX)               3
(BZ=F, BZ=F)               2
(IRFM11.SA, IRFM11.SA)    37
dtype: int64


In [13]:
Features_B = Features_B.ffill()
Features_B = Features_B.dropna()

print(f"Total de NaNs: {Features_B.isna().sum().sum()}")

Total de NaNs: 0


In [14]:
Features_B.columns

Features_B.columns = [
    'scoreSemanal',
    'ipcaMensal',
    'selicMeta',
    'ibcBrActivity',
    'dolarPtax',
    'sp500Index',
    'vixIndex',
    'treasuryYield10y',
    'brentOilPrice',
    'usdBrlExchange',
    'irfm11FixedIncome',
    'IPCA_Ano_Atual'
]

Features_B.columns

Index(['scoreSemanal', 'ipcaMensal', 'selicMeta', 'ibcBrActivity', 'dolarPtax',
       'sp500Index', 'vixIndex', 'treasuryYield10y', 'brentOilPrice',
       'usdBrlExchange', 'irfm11FixedIncome', 'IPCA_Ano_Atual'],
      dtype='object')

In [15]:
print(Features_B)

            scoreSemanal  ipcaMensal  selicMeta  ibcBrActivity  dolarPtax  \
2019-09-27      6.619598       -0.04        5.5       96.89367     4.1587   
2019-10-04      3.659769        0.10        5.5      100.32632     4.0610   
2019-10-11      4.337313        0.10        5.5      100.32632     4.1060   
2019-10-18      6.581211        0.10        5.5      100.32632     4.1376   
2019-10-25      5.411805        0.10        5.5      100.32632     4.0133   
...                  ...         ...        ...            ...        ...   
2025-08-29      2.698916       -0.11       15.0      110.42518     5.4264   
2025-09-05      1.993150        0.48       15.0      109.44199     5.3968   
2025-09-12      2.419663        0.48       15.0      109.44199     5.3677   
2025-09-19      3.556344        0.48       15.0      109.44199     5.3276   
2025-09-26      4.593504        0.48       15.0      109.44199     5.3445   

             sp500Index   vixIndex  treasuryYield10y  brentOilPrice  \
2019

Pre-processamento antes do XGBOOST

In [16]:
padrao = pd.read_csv('./dados/BaseA.csv')
padrao['Data'] = pd.to_datetime(padrao['Data'])
alinhamento = padrao['Data'].values

if not isinstance(Features_B.index, pd.DatetimeIndex):
    Features_B.index = pd.to_datetime(Features_B.index)

Features_B = Features_B.ffill()
Features_B = Features_B.dropna()

ativos = pd.read_csv('ibovespa.csv', sep=';')
dataset = {}
tickers = ativos['ticker']

print(f"Iniciando processamento para {len(tickers)} ativos...")

for ticker in tickers:
    try:
        df = yf.download(
            ticker,
            start=DATA_INICIAL,
            end=DATA_FIM,
            auto_adjust=True,
            progress=False
        )
        
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df = df[df.index.weekday == 4]

        var = pd.merge(df, Features_B, left_index=True, right_index=True, how='inner')

        var['AlvoRetorno'] = var['Close'].shift(-1) / var['Close'] - 1
        var['Alvo'] = np.where(var['AlvoRetorno'] > 0, 1, 0)

        var = var.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], errors='ignore')
        
        var = var.dropna(subset=['Alvo', 'AlvoRetorno'])
        var['Alvo'] = var['Alvo'].astype(int)

        var_final = var[var.index.isin(alinhamento)].copy()

        if not var_final.empty:
            dataset[ticker] = var_final
            
    except Exception as e:
        pass

Iniciando processamento para 59 ativos...


In [17]:
import pandas as pd

ref = pd.read_csv('./dados/Interseccao.csv')
DatasEmp = pd.to_datetime(ref['Data']).values

if ticker in dataset:
    dataset[ticker] = dataset[ticker][dataset[ticker].index.isin(DatasEmp)].copy()

In [18]:
import numpy as np

for tick, df in dataset.items():
    featuresCol = [c for c in df.columns if c not in ['Alvo', 'AlvoRetorno','scoreSemanal']]
    
    x = df[featuresCol]
    
    corr_matrix = x.corr().abs()

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    drop = [column for column in upper.columns if any(upper[column] > 0.95)]

    if len(drop) > 0:
        print(f"[{tick}] Removendo {len(drop)} features redundantes: {drop}")
        
        dataset[tick] = df.drop(columns=drop)
print("\nTratamento de correlação concluido")

[PETR4.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[ITUB4.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[VALE3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BPAC11.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[ABEV3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[WEGE3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BBDC3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[ITSA4.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BBAS3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[SANB11.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[VIVT3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[SBSP3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BBSE3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[B3SA3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[SUZB3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[CPFE3.SA] Removendo 1 features redund

Teste de datas iguais em ambos os modelos

In [19]:
print(dataset['PETR4.SA'].tail(-10))

            scoreSemanal  ipcaMensal  selicMeta  ibcBrActivity  dolarPtax  \
2019-12-13      4.431890        1.15        4.5       96.56505     4.0949   
2019-12-20      5.849578        1.15        4.5       96.56505     4.0777   
2019-12-27      2.561557        1.15        4.5       96.56505     4.0545   
2020-01-03      0.724997        0.21        4.5       94.02696     4.0522   
2020-01-10      4.051150        0.21        4.5       94.02696     4.0745   
...                  ...         ...        ...            ...        ...   
2025-08-22      3.052707       -0.11       15.0      110.42518     5.4392   
2025-08-29      2.698916       -0.11       15.0      110.42518     5.4264   
2025-09-05      1.993150        0.48       15.0      109.44199     5.3968   
2025-09-12      2.419663        0.48       15.0      109.44199     5.3677   
2025-09-19      3.556344        0.48       15.0      109.44199     5.3276   

             sp500Index  vixIndex  treasuryYield10y  brentOilPrice  \
2019-

In [20]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

modelos = {}

for ticker, df in dataset.items():
    print(f"\n--- Treinando: {ticker} ---")
    
    features = [c for c in df.columns if c not in ['Alvo', 'AlvoRetorno']]
    X = df[features]
    y = df['Alvo']

    split = int(len(df) * 0.8)
    
    X_dev = X.iloc[:split]
    y_dev = y.iloc[:split]
    
    X_backtest = X.iloc[split:]
    y_backtest = y.iloc[split:]
    
    def objective(trial):

        cutoff = int(len(X_dev) * 0.75)
        X_train_opt, X_val_opt = X_dev.iloc[:cutoff], X_dev.iloc[cutoff:]
        y_train_opt, y_val_opt = y_dev.iloc[:cutoff], y_dev.iloc[cutoff:]
        
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
            'gamma': trial.suggest_float('gamma', 0.0, 5.0),
            'random_state': 42,
            'n_jobs': -1
        }
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train_opt, y_train_opt)
        
        preds = model.predict(X_val_opt)
        
        score = precision_score(y_val_opt, preds, zero_division=0)
        
        return score

    study = optuna.create_study(direction='maximize')
    
    study.optimize(objective, n_trials=100) 
    
    print(f"Melhor Score: {study.best_value:.2%}")
    print(f"Melhores Parâmetros: {study.best_params}")

    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['n_jobs'] = -1
    
    final_model = xgb.XGBClassifier(**best_params)
    final_model.fit(X_dev, y_dev)
    
    modelos[ticker] = final_model



--- Treinando: PETR4.SA ---
Melhor Score: 100.00%
Melhores Parâmetros: {'n_estimators': 352, 'max_depth': 7, 'learning_rate': 0.15656253609390708, 'subsample': 0.6326402922731489, 'colsample_bytree': 0.6616779426829875, 'reg_alpha': 1.9644902934479518, 'reg_lambda': 7.0365103897891474, 'gamma': 4.580678503705917}

--- Treinando: ITUB4.SA ---
Melhor Score: 75.00%
Melhores Parâmetros: {'n_estimators': 241, 'max_depth': 10, 'learning_rate': 0.16518120022685034, 'subsample': 0.6964300280037468, 'colsample_bytree': 0.8072259006275974, 'reg_alpha': 4.391318982805717, 'reg_lambda': 6.525262122371452, 'gamma': 0.9300066396197588}

--- Treinando: VALE3.SA ---
Melhor Score: 100.00%
Melhores Parâmetros: {'n_estimators': 393, 'max_depth': 9, 'learning_rate': 0.14373456115138894, 'subsample': 0.7078210268832331, 'colsample_bytree': 0.6681639323869744, 'reg_alpha': 0.7968543595266717, 'reg_lambda': 4.027227949295552, 'gamma': 1.030582641146247}

--- Treinando: BPAC11.SA ---
Melhor Score: 57.69%
Mel

In [21]:
joblib.dump(modelos, '../modelos/ModelosB.pkl')
print("Arquivo mestre salvo com sucesso.")

Arquivo mestre salvo com sucesso.


In [19]:
treino = {}

for ticker, df in dataset.items():

    split = int(len(df) * 0.8)
    
    dfcut = df.iloc[split:].copy()
    
    treino[ticker] = dfcut

joblib.dump(treino, '../FeaturesB.pkl')

['../FeaturesB.pkl']